## Table of Contents
- [Question 2](#section-one)
- [Gene Association File (GAF)](#section-two)
- [Open Biomedical Ontologies](#section-three)
- [Map the Gene IDs to GO IDs, and GO IDs to GO terms](#section-four)
- [Output the results to a JSON file](#section-five)
- [Return to Project Table of Contents](project_overview.ipynb)

In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

data = pd.read_excel('Malaria_Research_Data.xlsx', header=0)
total_spectra = data.groupby('Biological sample category')['Protein percentage of total spectra'].sum()

# Normalize the protein percentages within each group
data['Normalized protein percentage'] = data.groupby('Biological sample category')['Protein percentage of total spectra'].transform(lambda x: (x / x.sum()) * 100)

print(data.columns)

Index(['Experiment name', 'Biological sample category', 'Protein group',
       'Protein accession number', 'Protein name',
       'Protein identification probability',
       'Protein percentage of total spectra', 'Number of unique peptides',
       'Number of unique spectra', 'Number of total spectra',
       'Peptide sequence', 'Previous amino acid', 'Next amino acid',
       'Peptide identification probability',
       'Modifications identified by spectrum', 'Peptide start index',
       'Peptide stop index', 'Unnamed: 17', 'Normalized protein percentage'],
      dtype='object')


In [8]:

# Filter proteins for SPP and NAT
spp_proteins = data[data['Biological sample category'] == 'SPP']
nat_proteins = data[data['Biological sample category'] == 'NAT']

# Calculate the mean normalized protein percentage for SPP and NAT
spp_mean_percentage = spp_proteins['Normalized protein percentage'].mean()
nat_mean_percentage = nat_proteins['Normalized protein percentage'].mean()

# Identify proteins that show up more in SPP than in NAT
enriched_proteins = spp_proteins[spp_proteins['Normalized protein percentage'] > nat_mean_percentage]

# Group the filtered data by protein name and calculate counts
protein_counts = enriched_proteins.groupby('Protein name').size().reset_index(name='Count')

# Sort the proteins based on counts in descending order
sorted_proteins = protein_counts.sort_values('Count', ascending=False)

# Create a list to store the unique GN names
unique_gn_list = []

# Print the count and names of the enriched proteins
print("Number of proteins showing up more in SPP than in NAT: {}".format(len(sorted_proteins)))
print("Protein names and their counts:")
for index, row in sorted_proteins.iterrows():
    protein_name = row['Protein name']
    count = row['Count']
    gn_index = protein_name.find('GN=')
    if gn_index != -1:
        gn_name = protein_name[gn_index+3:].split(' ')[0]
        unique_gn_list.append(gn_name)
        print("{}, Count: {}".format(gn_name, count))
    else:
        print("GN Name: Not found, Count: {}".format(count))


Number of proteins showing up more in SPP than in NAT: 52
Protein names and their counts:
PF3D7_1408600, Count: 200
PF3D7_0719600, Count: 167
PF3D7_0818900, Count: 137
PF3D7_1357000, Count: 120
PF3D7_0929400, Count: 117
PF3D7_1462800, Count: 105
PF3D7_1027300, Count: 94
PF3D7_1338200, Count: 84
PF3D7_1108700, Count: 81
PF3D7_0516200, Count: 78
PF3D7_0917900, Count: 73
PF3D7_0507100, Count: 69
PF3D7_0312800, Count: 68
PF3D7_1342000, Count: 66
PF3D7_0618300, Count: 65
PF3D7_1126200, Count: 64
PF3D7_1105400, Count: 55
MAL3P7.35, Count: 54
PF3D7_0519400, Count: 54
PF3D7_1451900, Count: 54
PF3D7_0708400, Count: 53
PF3D7_1242700, Count: 53
PF3D7_1142500, Count: 52
PF3D7_0629200, Count: 51
PF3D7_0813900, Count: 51
PF3D7_1347500, Count: 49
PF3D7_1445900, Count: 44
PF3D7_1437900, Count: 43
PF3D7_1104400, Count: 43
FBPA, Count: 42
PF3D7_1323400, Count: 42
PF3D7_1302800, Count: 42
PF3D7_0306900, Count: 41
PF3D7_1130100, Count: 41
PF3D7_1010700, Count: 41
PF3D7_1224300, Count: 39
PF3D7_1424100, Co

<a id="section-one"></a>
# Question 2: What functions are controlled by the unique things that they specific enzyme (SPP) can do?

### The question asked by the researchers requires us to take the Gene IDs found about to have been enriched in SPP vs NAT and discover the Gene Ontology (GO) terms associated with those genes. In order to accomplish this, we had to use two different research tools:

<a id="section-two"></a>
## Gene Association File (GAF)

### GAF Files contain specific information about genes and their characteristics. The GAF file used in our project is specific to _Plasmodium falciparum_ and contains several fields relating to this particular organism, including:

- DB Object ID: This correlates to the Gene IDs we identified above
- Gene Object Symbol: Which is often a more recognizable name for the gene
- GO ID: This is an identifier that can be associated to the specific functions that we are being asked about in the question

In [9]:
# Load the GAF file
gaf_file = 'genedb_pfalciparum.gaf'
gaf_df = pd.read_csv(gaf_file, delimiter='\t', comment='!', header=None)

<a id="section-three"></a>
## Open Biomedical Ontologies (OBO) File

### The OBO file that we are using has come from the Gene Ontology (GO) project. It provides a framework and hierarchy for annotating genes. This file contains both the GO ID (which we have retrieved from the GAF file) and a series of terms, called GO Terms, which are the answers to the researchers' questions!

In [10]:
# Load the OBO file
obo_file = 'go-basic.obo'

<a id="section-four"></a>
##  Map the Gene IDs to GO IDs, and GO IDs to GO terms

In [11]:
go_id_to_term = {}
current_go_id = None

with open(obo_file, 'r') as file:
    for line in file:
        line = line.strip()
        if line.startswith('id:'):
            current_go_id = line.split(' ')[1]
        elif line.startswith('name:'):
            go_term = line.split(' ', 1)[1]
            go_id_to_term[current_go_id] = go_term

# Extract gene IDs and related GO terms
gene_to_go_terms = {}
for index, row in gaf_df.iterrows():
    gene_id = row[1]
    go_id = row[4]
    go_term = go_id_to_term.get(go_id)

    if go_term and gene_id in unique_gn_list:
        if gene_id not in gene_to_go_terms:
            gene_to_go_terms[gene_id] = set()
        gene_to_go_terms[gene_id].add(go_term)

# Assign gene IDs and their related GO terms to a dictionary and print them
results = {}
for gene_id, go_terms in gene_to_go_terms.items():
    results[gene_id] = list(go_terms)
    print(f"Gene ID: {gene_id}")
    print("GO Terms:")
    for go_term in go_terms:
        print(go_term)
    print()


Gene ID: PF3D7_0306900
GO Terms:
structural constituent of ribosome
small ribosomal subunit
nucleus
ribosome
cytosolic small ribosomal subunit

Gene ID: PF3D7_0312800
GO Terms:
large ribosomal subunit
nucleus

Gene ID: PF3D7_0507100
GO Terms:
structural constituent of ribosome
translation
RNA binding
nucleus
ribosome
cytosolic large ribosomal subunit

Gene ID: PF3D7_0516200
GO Terms:
structural constituent of ribosome
translation
extracellular vesicle
RNA binding
nucleus
ribosome
cytosolic small ribosomal subunit

Gene ID: PF3D7_0519400
GO Terms:
structural constituent of ribosome
translation
RNA binding
nucleus
ribosome
cytosolic small ribosomal subunit

Gene ID: PF3D7_0519700
GO Terms:
RNA N6-methyladenosine methyltransferase complex
nucleus

Gene ID: PF3D7_0524000
GO Terms:
RNA N6-methyladenosine methyltransferase complex
food vacuole
protein domain specific binding
RNA binding
nucleus
NLS-bearing protein import into nucleus

Gene ID: PF3D7_0618300
GO Terms:
nucleus
ribosome
structu

<a id="section-five"></a>
## Output the results to a JSON file

In [12]:
import json

file_path = 'go_terms_output.json'
with open(file_path, 'w') as json_file:
    json.dump(results, json_file)

![json](images/json.png)